In [2]:
from keras.models import model_from_json
#이미지 생성용
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [3]:
#모델 로드
#json_file = open("./Models/AlexNet/model_alex_20181120_10_52_00.json", "r")
#json_file = open("./Models/GoogLeNet/20181216_14_38_44/model_googlenet_20181216_14_38_44.json", "r")
json_file = open("./Models/DenseNet/20181215_19_50_55/model_densenet_20181215_19_50_55.json", "r")
#json_file = open("./Models/ResNet/20181215_01_01_05/model_resnet_20181215_01_01_05.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Weight 로드
#loaded_model.load_weights("./Models/AlexNet/model_alex_weight20181120_10_52_00.h5")
#loaded_model.load_weights("./Models/GoogLeNet/20181216_14_38_44/weight_googlenet_20181217_04_01_45.h5")
loaded_model.load_weights("./Models/DenseNet/20181215_19_50_55/weight_densenet_20181216_13_02_07.h5")
#loaded_model.load_weights("./Models/ResNet/20181215_01_01_05/weight_resnet_20181215_19_19_26.h5")
print("Loaded model from disk")

# Compling CNN
loaded_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

Loaded model from disk


In [11]:
# 설정값들
IMAGE_SIZE = 128
IMAGE_DEPTH = 3
LABEL_NUM = 16
BATCH_SIZE = 32

#테스트셋 불러오기
note_name1 = "Himitsu_no_Door_kara_Ai_ni_Kite_Spectrogram"
note_name2 = "Kyoukai_no_Kanata_Spectrogram"
note_name3 = "Sedap_Malam_Spectrogram"
note_name4 = "Wishing_Spectrogram"
note_name5 = "Yuuki_no_Pendant_Spectrogram"

test_datagen = ImageDataGenerator()

test_set = test_datagen.flow_from_directory('./Dataset_Multi/Testset_HighAccuracy/' + note_name5,
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# Step 수 계산
test_step = int(len(test_set.classes) / BATCH_SIZE) + 1

# 모델 평가하기
print("-- Evaluate --")
scores = loaded_model.evaluate_generator(
            test_set, 
            steps = test_step)

print("%s: %.2f%%" %(loaded_model.metrics_names[1], scores[1]*100))

Found 1063 images belonging to 16 classes.
-- Evaluate --
acc: 31.89%


In [4]:
# 설정값들
IMAGE_SIZE = 128
IMAGE_DEPTH = 3
LABEL_NUM = 16
BATCH_SIZE = 32

# 가장 큰값 찾아서 반환
def GetBiggestIndex(datas):
    index = 0
    biggest_index = 0
    current_data = 0.0
    result = ""
    
    for data in datas:
        if current_data < data:
            current_data = data
            biggest_index = index
        index += 1
    
    for key, value in class_list.items():
        if value == biggest_index:
            result = key
        
    return result

In [7]:
#예측용 데이터 불러오기
note_name1 = "Himitsu_no_Door_kara_Ai_ni_Kite_Spectrogram"
note_name2 = "Kyoukai_no_Kanata_Spectrogram"
note_name3 = "Sedap_Malam_Spectrogram"
note_name4 = "Wishing_Spectrogram"
note_name5 = "Yuuki_no_Pendant_Spectrogram"

predict_datagen = ImageDataGenerator()
predict_set = predict_datagen.flow_from_directory('./Dataset_Multi/Testset_HighAccuracy/' + note_name5,
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# Step 수 계산
predict_step = int(len(predict_set.classes) / BATCH_SIZE) + 1
class_list = predict_set.class_indices

# 모델 예측하기
print("-- Predict and Making Note --")
output = loaded_model.predict_generator(predict_set, steps=predict_step)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

#예시
print(predict_set.class_indices)
print(output[0])

#파일 이름
file_names = predict_set.filenames
start_num = -15
end_num = -11

# 예측 + 노트생성
index = 0
result = dict()
for datas in output:
    label = GetBiggestIndex(datas)
    
    if not 'N' in label:
        #print("Timing: ", file_names[index][start_num:end_num], " Label: ", label)
        key = file_names[index][start_num:end_num]
        result[key] = label
        
    index += 1

#정렬해서 출력
for key in sorted(result):
    print("Timing: ", key, " Label: ", result[key])

Found 1063 images belonging to 16 classes.
-- Predict and Making Note --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.000 0.000 0.000 0.000 0.000 0.001 0.000 0.000 0.007 0.000 0.000 0.000
 0.000 0.000 0.008 0.984]
Timing:  0005  Label:  AD
Timing:  0006  Label:  BD
Timing:  0011  Label:  B
Timing:  0014  Label:  ABC
Timing:  0015  Label:  AC
Timing:  0016  Label:  A
Timing:  0021  Label:  B
Timing:  0026  Label:  B
Timing:  0027  Label:  D
Timing:  0029  Label:  ABCD
Timing:  0030  Label:  AC
Timing:  0031  Label:  D
Timing:  0032  Label:  ACD
Timing:  0035  Label:  BD
Timing:  0036  Label:  BCD
Timing:  0038  Label:  D
Timing:  0040  Label:  C
Timing:  0043  Label:  A
Timing:  0045  Label:  ACD
Timing:  0046  Label:  CD
Timing:  0047  Label:  D
Timing:  0050  Label:  D
Timing:  0059  Label:  ABD
Timing:  0060  Label:  D
Timing:  0062  Label:  A
Timing:  0065  Label:  A
Timing: